In [ ]:
from bs4 import BeautifulSoup

htmlFile = open("course_directory_winter.html", "r", encoding="utf8")
html = htmlFile.read()

soup = BeautifulSoup(html, 'html.parser')
tds = soup.find_all('td')
textFile = open("course_directory_winter.txt", "w", encoding="utf8")

for td in tds:
    td_text = td.get_text(separator="\n").strip()
    td_text = td_text.replace("\n", ", ").strip()
    td_text = ', '.join([item.strip() for item in td_text.split(',') if item.strip()])
    if td_text:
        textFile.write(td_text + "\n")

textFile.close()


In [47]:
textFile = open("course_directory_winter.txt","r",encoding="utf8")
lines = textFile.readlines()
textFile.close()



In [48]:
# first three letters of a course code are characters and the rest are numbers
# function that identifies if a string is a course code
def isCourseCode(string):
	string = string.replace(" ","")
	if len(string) == 6 and string[:3].isalpha() and string[3:].isdigit():
		return True
	
	return False

In [50]:
# testing the accuracy of the isCourseCode function
# this block of code will print all the course codes found in the unprocessed text file

textFile = open("course_directory_winter.txt","r",encoding="utf8")
courses = []

for line in lines:
	if isCourseCode(line.split(",")[0]):
		courses.append(line.split(",")[0])
# winter 329/331 course found
# winter 283/282
print(len(courses), "Course found!")
# print(courses)


306 Course found!


In [51]:
# converting the text file into a json file
# basic foundation for the json file

# each course from the txt file is added to the array as a string
# each string is a course that contains all the information about the course

courseArray=[]
course = ""
flag = True
for line in lines:
	line = line.replace("\t", "")
	if isCourseCode(line.split(",")[0]):
		courseArray.append(course)
		course= ""
	course+=line

# removing the first empty string from the array
courseArray = list(filter(lambda a: a.replace(" ","")!="",courseArray))





In [52]:
# this function will return the type of the string like course code, section, day, date, time, etc

def detectType(s:str):
	Days = ["Mon", "Tue","Wed","Thu","Fri","Sat","Sun"]
	if s == "Section":
		return "Section"
	elif s in Days:
		return "Day"
	elif "-" in s:
		return "Date"
	elif ":" in s:
		return "Time"
	elif s == "to":
		return "To"
	elif s.isdigit():
		return "Section No."
	else:

		return "Useless"

In [ ]:
# main txt to json logic starts here

import re
lines = []
Code = ''
Level = ''
Name = ''
isBiSem = ''
Credits = ''
Faculties = ''
Semester = ''
Prerequisite = ''
Description = ''
Sections = {}
SectionDict = {}
DayArray = [[],[]]
TimeArray = []
DateArray = []
activeSection = ''
activeDay = ''

oneCourseJson = {};
finalJson = []
for course in courseArray:
	Code = ''
	Level = ''
	Name = ''
	isBiSem = ''
	Credits = ''
	Faculties = ''
	Semester = ''
	Prerequisite = ''
	Description = ''
	Sections = {}
	SectionDict = {}
	DayArray = [[],[]]
	TimeArray = []
	DateArray = []
	activeSection = ''
	activeDay = ''
	lines = course.split("\n")
	lines = list(filter(lambda a: a.replace(" ","")!="",    lines))

	# print(lines)
	# Level = lines[0].split(",")[1]
	# Level = Level.strip(" ").strip("[").strip("]")
	# Name = lines[1]
	# Credits = lines[2]
	# Faculties = lines[3].split(",")
	# Semester = lines[4]
	# parts = lines[0].split(",")
	# Code = parts[0].strip()
	Code = lines[0].split(",")[0]
	Level = lines[1].strip(" []") if len(lines) > 1 else "Unknown"
	Name = lines[1] if len(lines) > 1 else "Unknown"
	Credits = lines[2] if len(lines) > 2 else "Unknown"
	Faculties = lines[3].split(",") if len(lines) > 3 else ["Not added"]
	Semester = lines[4] if len(lines) > 4 else "Unknown"
	

	# if there is no faculty added to the course then the course is not added to the json file
	if Faculties[0] == "Not added":
		continue

	# if there is a prerequisite then it is added to the json file then the index of the description is changed
	if lines[5].split(",")[0] == "PREQ_OR":
		Prerequisite = lines[5].split(",")[1]
		n = 5
	else:
		Prerequisite = None
		n = 4
	Description = lines[n+1]
	print(Code)
	for i in lines[n+2:]:
		i = i.strip(",")
		i = i.replace(" ",",")
		i = i.split(",")
		# print(Code)
		

		i = list(filter(lambda a: a!="Quarter]",i))
		i = list(filter(lambda a: a!="[First",i))
		i = list(filter(lambda a: a!="[Second",i))
		i = list(filter(lambda a: len(a )!=0,i))
		i = list(map(lambda a: a.replace("[",""),i))
		i = list(map(lambda a: a.replace("]",""),i))
		i = list(map(lambda a: a.strip(","),i))
		# print(i)
		try: 
			for j in i:
				
				j = j.strip(" ")
				typeOfJ = detectType(j)
				# print(j,typeOfJ)
				
				if j=="Bi-Semester":
					continue
				# print(j)
				if typeOfJ == "Section No.":
					activeSection = j
					Sections[j] = {}
				
				elif typeOfJ == "Day":
					activeDay = j
					if Sections[activeSection].get(activeDay) == None:
						Sections[activeSection][activeDay] = [[],[]]
					# Sections[activeSection][activeDay] = [[],[]]
				elif typeOfJ == "Time":
					# if Sections[activeSection][activeDay][0].count(j) == 0:
					Sections[activeSection][activeDay][0].append(j)
					# Sections[activeSection][activeDay][0].append(j)
				elif typeOfJ == "Date":
					# if Sections[activeSection][activeDay][1].count(j) == 0:
					Sections[activeSection][activeDay][1].append(j)
				elif typeOfJ == "To":
					pass
				elif typeOfJ == "Useless":
					pass

		except:
			# raise Exception("Error in course: "+Code)
			pass

	Sections.popitem() # remove other course's courseID to act as this course's section
	# print(Sections)

	oneCourseJson = {
		"Code":Code,
		"Level":Level,
		"Name":Name,
		"Credits":Credits,
		"Faculties":Faculties,
		"Semester":Semester,
		"Prerequisite":Prerequisite,
		"Description":Description,
		"Sections":Sections
	}
	finalJson.append(oneCourseJson)

import json
with open('winter.json', 'w') as outfile:
	json.dump(finalJson, outfile, indent=4)



	


	# print(lines[5], lines[6],12)
	# print(Code, Level, Name, Credits, Faculties,Semester, Prerequisite, Description)

COM101
{'1': {'Mon': [['08:00', '09:30'], ['06-01-2025', '18-04-2025']], 'Fri': [['08:00', '09:30'], ['06-01-2025', '18-04-2025']]}, '2': {'Tue': [['08:00', '09:30'], ['06-01-2025', '18-04-2025']], 'Thu': [['08:00', '09:30'], ['06-01-2025', '18-04-2025']]}, '3': {'Mon': [['16:00', '17:30'], ['06-01-2025', '18-04-2025']], 'Fri': [['16:00', '17:30'], ['06-01-2025', '18-04-2025']]}, '4': {'Tue': [['16:00', '17:30'], ['06-01-2025', '18-04-2025']], 'Thu': [['16:00', '17:30'], ['06-01-2025', '18-04-2025']]}}
FAC331
{'1': {'Mon': [['14:30', '16:00'], ['06-01-2025', '18-04-2025']], 'Fri': [['14:30', '16:00'], ['06-01-2025', '18-04-2025']]}}
MGT136
{'1': {'Mon': [['11:00', '12:30'], ['06-01-2025', '28-02-2025']], 'Wed': [['11:00', '12:30'], ['06-01-2025', '28-02-2025']]}}
MKT341
{'1': {'Tue': [['13:00', '14:30'], ['06-01-2025', '18-04-2025']], 'Thu': [['13:00', '14:30'], ['06-01-2025', '18-04-2025']]}}
MGT105
{'1': {'Mon': [['17:30', '19:00'], ['06-01-2025', '18-04-2025']], 'Fri': [['17:30', '1